In [ ]:
# install dependencies, this requires Python 3.11 or higher
!pip3 install snowplow-signals==0.4.0

For Snowplow Signals (BDP) you will need
* API_URL - The URL for your Signals instance
* API_KEY - the API key generated in console
* API_KEY_ID - the API key ID generated in console
* ORG_ID - your organization ID from console (found in the settings page)

Otherwise for Signals Sandbox you will need
* API_URL - The URL for your Signals instance
* ACCESS_TOKEN - The access token (on the dashboard page)

In [ ]:
from snowplow_signals import Criteria, Criterion, Signals, Attribute, AtomicProperty, Event, EntityProperty, EventProperty, domain_sessionid, SignalsSandbox

# if you are using Signals BDP set the following variables
API_URL = 'example.signals.snowplowanalytics.com'
API_KEY = ''
API_KEY_ID = ''
ORG_ID = ''

# or if you are using Signals Sandbox set the following variables instead
API_URL = 'you.signals.snowplowanalytics.com'
ACCESS_TOKEN = ''


sandbox = ACCESS_TOKEN is not None and ACCESS_TOKEN != ''

if not sandbox:
    print('Using Snowplow Signals')
    sp_signals = Signals(
        api_url=API_URL,
        api_key=API_KEY,
        api_key_id=API_KEY_ID,
        org_id=ORG_ID
    )
else:
    print('Using Signals Sandbox')
    sp_signals = SignalsSandbox(
        api_url=API_URL,
        sandbox_token=ACCESS_TOKEN
    )

Now that we have our connection setup to Signals we can define a set of attributes based on the events in our travel app.

In [ ]:
# now let's define some attributes using the connection from above

travel_vendor = 'com.snowplowanalytics.accelerators.travel'

# tags pertaining to different segments
cultural_explorer_tags = ["culture", "history", "heritage", "ancient", "temples", "art", "traditional"]
modern_urbanite_tags = ["urban", "nightlife", "shopping", "modern", "architecture"]
tranquil_seeker_tags = ["nature", "peaceful", "wellness", "beaches", "mountains", "river", "wellness"]
family_fun_tags = ["family-friendly", "beaches", "nature", "food", "mountains", "culture"]
culinary_tourist_tags = ["food", "street food", "multicultural", "traditional", "urban", "shopping"]


page_view_count = Attribute(
    name="page_view_count",
    type="int32",
    description="A count of all page view events",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0"
        )
    ],
    aggregation="counter"
)

dest_page_view_count = Attribute(
    name="destination_page_view_count",
    type="int32",
    description="A count of destination page view events",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0"
        )
    ],
    criteria=Criteria(
    all=[
        Criterion.like(
            property=AtomicProperty(name="page_urlpath"),
            value="%destinations%"
        )
    ]
    ),
    aggregation="counter"
)


family_destination_count = Attribute(
    name="family_destination_count",
    type="int32",
    description="A count of all interactions with family destination tags and pages.",
    events=[
        Event(
            vendor=travel_vendor,
            name="filter_tag_applied",
            version="1-0-0"
        ),
        Event(
            vendor='com.snowplowanalytics.snowplow',
            name='page_view',
            version="1-0-0"
        )
    ],
    aggregation="counter",
    criteria=Criteria(
        any=[
            (
                Criterion.eq(
                    property=EventProperty(
                        vendor=travel_vendor,
                        name="filter_tag_applied",
                        major_version=1,
                        path="tag_name"
                    ),
                    value="Family"
                )
            ),
            (
                Criterion.in_list(
                    property=EntityProperty(
                        vendor=travel_vendor,
                        name="content",
                        major_version=1,
                        path="primary_tag"
                    ),
                    values=family_fun_tags
                )
            )
        ]
    )
)

cultural_explorer = Attribute(
    name="cultural_explorer",
    type="int32",
    events=[
        Event(
            vendor=travel_vendor,
            name="filter_tag_applied",
            version="1-0-0",
        ),
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0"
        )
    ],
    aggregation="counter",
    criteria=Criteria(
        any=[
            (
                Criterion.in_list(
                    property=EventProperty(
                        vendor=travel_vendor,
                        name="filter_tag_applied",
                        major_version=1,
                        path="tag_name"
                    ),
                    values=cultural_explorer_tags
                )
            ),
            (
                Criterion.in_list(
                    property=EntityProperty(
                        vendor=travel_vendor,
                        name="content",
                        major_version=1,
                        path="primary_tag"
                    ),
                    values=cultural_explorer_tags
                )
            )
        ]
    )
)

modern_urbanite = Attribute(
    name="modern_urbanite",
    type="int32",
    events=[
        Event(
            vendor=travel_vendor,
            name="filter_tag_applied",
            version="1-0-0",
        ),
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0"
        )
    ],
    aggregation="counter",
    criteria=Criteria(
        any=[
            (
                Criterion.in_list(
                    property=EventProperty(
                        vendor=travel_vendor,
                        name="filter_tag_applied",
                        major_version=1,
                        path="tag_name"
                    ),
                    values=modern_urbanite_tags
                )
            ),
            (
                Criterion.in_list(
                    property=EntityProperty(
                        vendor=travel_vendor,
                        name="content",
                        major_version=1,
                        path="primary_tag"
                    ),
                    values=modern_urbanite_tags
                )
            )
        ]
    )
)

tranquil_seeker = Attribute(
    name="tranquil_seeker",
    type="int32",
    events=[
        Event(
            vendor=travel_vendor,
            name="filter_tag_applied",
            version="1-0-0",
        ),
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="counter",
    criteria=Criteria(
        any=[
            (
                Criterion.in_list(
                    property=EventProperty(
                        vendor=travel_vendor,
                        name="filter_tag_applied",
                        major_version=1,
                        path="tag_name"
                    ),
                    values=tranquil_seeker_tags
                )
            ),
            (
                Criterion.in_list(
                    property=EntityProperty(
                        vendor=travel_vendor,
                        name="content",
                        major_version=1,
                        path="primary_tag"
                    ),
                    values=tranquil_seeker_tags
                )
            )
        ]
    )
)

family_fun = Attribute(
    name="family_fun",
    type="int32",
    events=[
        Event(
            vendor=travel_vendor,
            name="filter_tag_applied",
            version="1-0-0",
        ),
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="counter",
    criteria=Criteria(
        any=[
            (
                Criterion.in_list(
                    property=EventProperty(
                        vendor=travel_vendor,
                        name="filter_tag_applied",
                        major_version=1,
                        path="tag_name"
                    ),
                    values=family_fun_tags
                )
            ),
            (
                Criterion.in_list(
                    property=EntityProperty(
                        vendor=travel_vendor,
                        name="content",
                        major_version=1,
                        path="primary_tag"
                    ),
                    values=family_fun_tags
                )
            )
        ]
    )
)


culinary_tourist = Attribute(
    name="culinary_tourist",
    type="int32",
    events=[
        Event(
            vendor=travel_vendor,
            name="filter_tag_applied",
            version="1-0-0",
        ),
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],

    aggregation="counter",
    criteria=Criteria(
        any=[
            (
                Criterion.in_list(
                    property=EventProperty(
                        vendor=travel_vendor,
                        name="filter_tag_applied",
                        major_version=1,
                        path="tag_name"
                    ),
                    values=culinary_tourist_tags
                )
            ),
            (
                Criterion.in_list(
                    property=EntityProperty(
                        vendor=travel_vendor,
                        name="content",
                        major_version=1,
                        path="primary_tag"
                    ),
                    values=culinary_tourist_tags
                )
            )
        ]
    )
)

preferred_experience_length = Attribute(
    name="preferred_experience_length",
    type="string",
    events=[
        Event(
            vendor="com.snowplowanalytics.accelerators.travel",
            name="experience_filter",
            version="1-0-0",
        )
    ],
    property=EventProperty(
        vendor="com.snowplowanalytics.accelerators.travel",
        name="experience_filter",
        major_version=1,
        path="filter_value"
    ),
    aggregation="last",
    criteria=Criteria(
        all=[
            (
                Criterion.eq(
                    property=EventProperty(
                        vendor="com.snowplowanalytics.accelerators.travel",
                        name="experience_filter",
                        major_version=1,
                        path="filter_type"
                    ),
                    value="duration"
                )
            )
        ]
    )
)

budget_conscious = Attribute(
    name="budget_conscious_count",
    type="int32",
    events=[
        Event(
            vendor="com.snowplowanalytics.accelerators.travel",
            name="sort_change",
            version="1-0-0",
        ),
        Event(
            vendor="com.snowplowanalytics.accelerators.travel",
            name="destination_filter",
            version="1-0-0",
        ),
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="counter",
    criteria=Criteria(
        any=[
            (
                Criterion.eq(
                    property=EventProperty(
                        vendor="com.snowplowanalytics.accelerators.travel",
                        name="sort_change",
                        major_version=1,
                        path="sort_option"
                    ),
                    value="price-low"
                )
            ),
            (
                Criterion.eq(
                    property=EventProperty(
                        vendor="com.snowplowanalytics.accelerators.travel",
                        name="destination_filter",
                        major_version=1,
                        path="filter_value"
                    ),
                    value="budget"
                )
            ),
            (
                Criterion.in_list(
                    property=EntityProperty(
                        vendor=travel_vendor,
                        name="content",
                        major_version=1,
                        path="primary_tag"
                    ),
                    values=["budget", "budget-friendly", "affordable"]
                )
            )
        ]
    )
)

luxury_inclined = Attribute(
    name="luxury_inclined_count",
    type="int32",
    events=[
        Event(
            vendor="com.snowplowanalytics.accelerators.travel",
            name="sort_change",
            version="1-0-0",
        ),
        Event(
            vendor="com.snowplowanalytics.accelerators.travel",
            name="destination_filter",
            version="1-0-0",
        ),
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="counter",
    criteria=Criteria(
        any=[
            (
                Criterion.eq(
                    property=EventProperty(
                        vendor="com.snowplowanalytics.accelerators.travel",
                        name="sort_change",
                        major_version=1,
                        path="sort_option"
                    ),
                    value="price-high"
                )
            ),
            (
                Criterion.eq(
                    property=EventProperty(
                        vendor="com.snowplowanalytics.accelerators.travel",
                        name="destination_filter",
                        major_version=1,
                        path="filter_value"
                    ),
                    value="luxury"
                )
            ),
            (
                Criterion.in_list(
                    property=EntityProperty(
                        vendor=travel_vendor,
                        name="content",
                        major_version=1,
                        path="primary_tag"
                    ),
                    values=["luxury", "high-end"]
                )
            )
        ]
    )
)

latest_schedule = Attribute(
    name="latest_schedule",
    type="string",
    events=[
        Event(
            vendor="com.snowplowanalytics.accelerators.travel",
            name="schedule_update",
            version="1-0-0",
        )
    ],
    aggregation="last",
    property=EventProperty(
        vendor="com.snowplowanalytics.accelerators.travel",
        name="schedule_update",
        major_version=1,
        path="schedule"
    )
)

We will want to fetch all of these attributes at once from the API so to do so we can create a logical container called an attribute group. Specifically we will use a StreamAttributeGroup as these features will all be calculated in real time.

In [ ]:
from snowplow_signals import StreamAttributeGroup

# next, we'll group these attributes together so we can retrieve them all at once
# into 'travel_view'. We will use our session id ('domain_userid') as our attribute key
# this effectively means these attributes will be aggregated at the session level
# however you can specify other keys here too if required (e.g., at the user level)

session_attributes_group = StreamAttributeGroup(
    name="travel_view",
    version=1,
    attribute_key=domain_sessionid,
    attributes=[page_view_count, dest_page_view_count, family_destination_count, cultural_explorer,
                modern_urbanite, tranquil_seeker, family_fun, culinary_tourist,
                preferred_experience_length, budget_conscious, luxury_inclined, latest_schedule],
                # + summary_attributes,
    owner = 'you@email.com',
)

# and finally define a service that exposes the group above, so we can access it via API



Now that we have this grouping we will create a service in order to expose this information via the API so we can fetch it in our site.

In [ ]:
from snowplow_signals import Service

travel_service = Service(
    name="travel_service",
    description="A service for our travel demo website.",
    attribute_groups=[session_attributes_group],
    owner='you@email.com'
)

# publish both the attribute group and the service to our API endpoint
response = sp_signals.publish([session_attributes_group, travel_service])
print(response)

For the final optional part of this tutorial we will create an intervention that invokes the agent when the user has browsed more than 3 different destinations.

In [ ]:
from snowplow_signals import RuleIntervention, InterventionCriterion, LinkAttributeKey

destination_assistance_intervention = RuleIntervention(
    name="destination_help",
    owner="you@email.com",
    description="Assist the user if they are looking at 3 or more destinations in a given session.",
    target_attribute_keys=[LinkAttributeKey(name="domain_sessionid")],
    version=1,
    criteria=InterventionCriterion(
        attribute="travel_view:destination_page_view_count",
        operator=">=",
        value=3,
    ),
)

sp_signals.publish([destination_assistance_intervention])